In [43]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [34]:
subset = pd.read_pickle("../output/final_subset_BnW.pkl")

In [35]:
subset.head()

,label,image_array
0,MSI,"[[154, 179, 144, 124, 152, 146, 167, 201, 180,..."
1,MSI,"[[160, 146, 134, 96, 67, 69, 78, 96, 124, 155,..."
2,MSI,"[[246, 241, 236, 216, 173, 164, 158, 162, 164,..."
3,MSI,"[[123, 151, 178, 134, 135, 139, 142, 139, 102,..."
4,MSI,"[[128, 160, 172, 204, 192, 147, 146, 136, 127,..."


In [36]:
X = np.asarray(list(subset["image_array"]))

In [37]:
X.shape

(2000, 224, 224)

In [38]:
subset["label"] = pd.Categorical(subset["label"], categories=["MSI","MSS"], ordered = True)
y = subset["label"].cat.codes
y.value_counts()

1    1000
0    1000
dtype: int64

In [39]:
print(X.shape, y.shape)

(2000, 224, 224) (2000,)


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [44]:
# Prepare data to feed the NN
num_classes = 2
img_rows, img_cols = 224, 224

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (1600, 224, 224, 1)
1600 train samples
400 test samples


In [46]:
## This is the neural network proposed architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [47]:
# Fit the NN
batch_size = 20
epochs = 10

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1, validation_data=(X_test, y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 230s 143ms/step - loss: 0.6935 - accuracy: 0.5050 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 2/10
1600/1600 [==============================] - 218s 136ms/step - loss: 0.6933 - accuracy: 0.5038 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 3/10
1600/1600 [==============================] - 217s 136ms/step - loss: 0.6932 - accuracy: 0.4919 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 4/10
1600/1600 [==============================] - 214s 134ms/step - loss: 0.6934 - accuracy: 0.4781 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 5/10
1600/1600 [==============================] - 212s 133ms/step - loss: 0.6932 - accuracy: 0.4900 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 6/10
1600/1600 [==============================] - 214s 133ms/step - loss: 0.6933 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 7/10
1600/1600 [==============================] - 214s 134m